<a href="https://colab.research.google.com/github/brayvid/efficient-portfolio/blob/main/EfficientPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Efficient Portfolio Construction</h1>
<h4>Blake Rayvid - <a href=https://github.com/brayvid>https://github.com/brayvid</a></h4>

In *An Analytic Derivation of the Efficient Portfolio Frontier* (1972), [Robert C. Merton](https://en.wikipedia.org/wiki/Robert_C._Merton) described an algorithm to construct a portfolio with the <ins>smallest variance in expected returns for a given level of expected returns</ins> (efficient). The algorithm takes as input a list of candidate securities and their historical annual returns as decimal values, and it outputs the fraction of the portfolio to be allocated to each. Each of these output percentages may be positive or negative corresponding to long and short positions, or zero when no position should be taken, but they will sum to 100%. The example below generates an allocation strategy using the whole S&P 100.

In [31]:
# -*- coding: utf-8 -*-
# --- Final, Future-Proof, Single-Block Script ---

# --- 1. Imports and Setup ---
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import date

# --- 2. Algorithm/Program Inputs ---
E = 0.25 # Target annual return
START_YEAR = 2010
# Set the desired end year. The script will automatically adjust it if it's the current (incomplete) year.
END_YEAR = 2025 # <-- Simulating that the current year is 2025

# --- 3. Fetch S&P 100 Tickers ---
print("Fetching S&P 100 ticker list from Wikipedia...")
try:
    url = 'https://en.wikipedia.org/wiki/S%26P_100'
    tables = pd.read_html(url)
    tickers = tables[2]['Symbol'].tolist()
    tickers = [ticker.replace('-', '.') for ticker in tickers]
    print(f"Successfully fetched {len(tickers)} tickers.")
except Exception as e:
    print(f"Could not fetch tickers from Wikipedia: {e}")
    tickers = []

# --- 4. Fetch and Prepare Data ---
if tickers:
    # --- DYNAMIC DATE CORRECTION LOGIC ---
    # This logic makes the script future-proof.
    # We will simulate that today's date is in 2025 for this example.

    # In a real scenario, this would be: today_year = date.today().year
    today_year = 2025 # <-- SIMULATING THE CURRENT YEAR

    last_full_year = today_year - 1

    if END_YEAR >= today_year:
        print(f"\n--- IMPORTANT DATA ADJUSTMENT ---")
        print(f"Warning: The provided end year ({END_YEAR}) is the current (incomplete) year or in the future.")
        print(f"Using last full year ({last_full_year}) for calculations to ensure data quality.")
        print(f"----------------------------------\n")
        END_YEAR = last_full_year
    # --- END OF DYNAMIC LOGIC ---

    start_date = f"{START_YEAR}-01-01"
    end_date = f"{END_YEAR}-12-31"

    print(f"Downloading daily price data from {start_date} to {end_date}...")
    price_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=True)

    valid_tickers = [t for t in tickers if t in price_data.columns.get_level_values(0) and not price_data[t].empty]
    close_price_series = [price_data[ticker]['Close'].rename(ticker) for ticker in valid_tickers]
    close_prices = pd.concat(close_price_series, axis=1)

    print(f"Successfully processed price data for {len(close_prices.columns)} tickers.")

    print("Calculating annual returns...")
    annual_returns = close_prices.resample('YE').last().pct_change()

    annual_returns = annual_returns.dropna(how='all', axis=0)
    original_cols = len(annual_returns.columns)
    data = annual_returns.dropna(axis=1).T
    data.columns = data.columns.year

    print(f"Data cleaned. Removed {original_cols - len(data.T.columns)} stocks with incomplete data.")
    print(f"Using {len(data)} stocks for analysis.")

    # --- 5. Run Merton's Algorithm ---
    R = data.values
    symbs = data.index
    avg_ret = np.mean(R, axis=1)
    cov_mat = np.cov(R)
    inv_cov = np.linalg.inv(cov_mat)
    ones = np.ones(len(avg_ret))
    a = ones.T @ inv_cov @ avg_ret
    b = avg_ret.T @ inv_cov @ avg_ret
    c = ones.T @ inv_cov @ ones
    d = b * c - a**2
    g_vector = (1/d) * (b * (inv_cov @ ones) - a * (inv_cov @ avg_ret))
    h_vector = (1/d) * (c * (inv_cov @ avg_ret) - a * (inv_cov @ ones))
    X = g_vector + h_vector * E

    # --- 6. Analyze and Display Results ---
    results_df = pd.DataFrame({'Allocation': X}, index=symbs)
    portfolio_return = results_df['Allocation'].values @ avg_ret
    portfolio_volatility = np.sqrt(results_df['Allocation'].values.T @ cov_mat @ results_df['Allocation'].values)

    print("\n--- Efficient Portfolio Analysis ---")
    print(f"Target Annual Return: {E:.1%}")
    print("\n--- Calculated Portfolio Characteristics ---")
    print(f"Expected Annual Return:    {portfolio_return:.1%}")
    print(f"Expected Annual Volatility (Risk): {portfolio_volatility:.1%}")
    sum_of_weights = results_df['Allocation'].sum()
    print(f"Sum of All Weights:        {sum_of_weights:.4%}")

    results_df_sorted = results_df.sort_values(by='Allocation', ascending=False)
    print("\n--- Top 5 Long Positions ---")
    print(results_df_sorted.head(5).to_string(formatters={'Allocation': '{:,.2%}'.format}))
    print("\n--- Top 5 Short Positions ---")
    print(results_df_sorted.tail(5).to_string(formatters={'Allocation': '{:,.2%}'.format}))

    outString = "sp100_alloc_future_proof"
    file_name = f"./{outString}_E={E*100:.0f}%.csv"
    results_df_sorted.to_csv(file_name)
    print(f"\nFull portfolio allocation saved to: {file_name}")

else:
    print("\nCould not fetch data. Halting execution.")

Fetching S&P 100 ticker list from Wikipedia...
Successfully fetched 101 tickers.

--- IMPORTANT DATA ADJUSTMENT ---
Using last full year (2024) for calculations to ensure data quality.
----------------------------------



/tmp/ipython-input-1467679636.py:51: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=True)
[*********************100%***********************]  101 of 101 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Successfully processed price data for 101 tickers.
Calculating annual returns...
Data cleaned. Removed 6 stocks with incomplete data.
Using 95 stocks for analysis.

--- Efficient Portfolio Analysis ---
Target Annual Return: 25.0%

--- Calculated Portfolio Characteristics ---
Expected Annual Return:    27.0%
Expected Annual Volatility (Risk): 0.0%
Sum of All Weights:        100.0000%

--- Top 5 Long Positions ---
     Allocation
MRK      86.09%
ACN      57.91%
VZ       51.37%
PM       47.32%
SBUX     45.47%

--- Top 5 Short Positions ---
     Allocation
SO      -42.34%
PG      -42.40%
MDLZ    -47.41%
HON     -47.77%
CL      -50.24%

Full portfolio allocation saved to: ./sp100_alloc_future_proof_E=25%.csv


/tmp/ipython-input-1467679636.py:60: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  annual_returns = close_prices.resample('YE').last().pct_change()
